In [1]:
#import packages: pandas, numpy, pyplot, glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob 

# Arrival Data

In [2]:
#use glob to generate list of arrival detail files
arr_stat_list = glob.glob("Detailed*.csv")
arr_stat_list

['Detailed_Statistics_Arrivals_AA.csv',
 'Detailed_Statistics_Arrivals_AS.csv',
 'Detailed_Statistics_Arrivals_B6.csv',
 'Detailed_Statistics_Arrivals_DL.csv',
 'Detailed_Statistics_Arrivals_F9.csv',
 'Detailed_Statistics_Arrivals_UA.csv',
 'Detailed_Statistics_Arrivals_WN.csv']

In [3]:
#read in CSVs into dummy list
cols = [0, 1, 4, 5, 7, 8, 9]
df_list = []

for i in range(7):
    file = arr_stat_list[i]
    df_list.append(pd.read_csv(file, 
                engine = 'python',
                usecols = cols, 
                dtype = {'Origin Airport':'category'},
                skiprows = 7, 
                parse_dates = [[1, 3]],
                skipfooter = 1
                ))

In [4]:
#concatenate list into single df, reset index, drop original index, rename cols, 
#and make the carrier and origin columns categoricals
arr_data = pd.concat(df_list)
arr_data.reset_index(inplace = True)
arr_data = arr_data.drop("index", axis = 1)
arr_data.columns = ['scheduled_arr', 'carrier', 'origin', 'scheduled_elapsed', 'actual_elapsed', 'arr_delay']
arr_data[['carrier', 'origin']] = arr_data[['carrier', 'origin']].astype('category')

In [5]:
#read in airport distance info
distance = pd.read_csv('SLC_routes.csv')
arr_data_dist = arr_data.merge(distance, how = 'left', left_on = 'origin', right_on = 'faa_code')
arr_data_dist.drop('faa_code', axis = 1)

,scheduled_arr,carrier,origin,scheduled_elapsed,actual_elapsed,arr_delay,distance
0,1988-01-01 11:02:00,AA,ORD,197,211,22,1245
1,1988-01-01 11:17:00,AA,DFW,160,170,9,987
2,1988-01-01 12:31:00,AA,DFW,154,174,21,987
3,1988-01-01 15:00:00,AA,JAC,52,50,-3,204
4,1988-01-01 07:50:00,AA,IDA,48,0,0,188
...,...,...,...,...,...,...,...
2043276,2019-12-31 11:30:00,WN,MDW,210,191,-9,1254
2043277,2019-12-31 15:45:00,WN,PHX,100,86,-16,507
2043278,2019-12-31 10:00:00,WN,LAX,110,118,-1,589
2043279,2019-12-31 22:20:00,WN,SAN,110,110,6,626


In [6]:
#add cols: 'arr_DateHour', 'date', 'day_name', and 'day_of_year' and 'sceduled_hour'
arr_data_dist['arr_DateHour'] = arr_data_dist['scheduled_arr'].dt.round('H')
arr_data_dist['date'] = arr_data_dist['scheduled_arr'].dt.date
arr_data_dist['day_name'] = arr_data_dist['scheduled_arr'].dt.day_name()
arr_data_dist['day_of_year'] = arr_data_dist['scheduled_arr'].dt.dayofyear
arr_data_dist['date'] = pd.to_datetime(arr_data_dist['date'])
arr_data_dist['scheduled_hour'] = arr_data_dist['scheduled_arr'].dt.hour
arr_data_dist['year'] = arr_data_dist['scheduled_arr'].dt.year

# Precipitation Data

In [7]:
#read in precip data
precip = pd.read_csv('kslc_precip_data.csv',
                    usecols = [2, 3])
precip["DATE"] = pd.to_datetime(precip["DATE"])

#create a dt index, resample the data to hourly, and replace NaN with 0
precip.set_index('DATE', inplace = True)
precip_hour = precip.resample('H').asfreq()
precip_hour.fillna(0, inplace = True)
precip_hour.reset_index(inplace = True)
precip_hour.columns = ['DateHour', 'HourPrecip']

# Weather Data

In [8]:
#read in weather data
weather = pd.read_csv('kslc_daily_weather_data.csv')
weather["date"] = pd.to_datetime(weather["date"])
weather.drop('wind_fastest_1min', axis = 1, inplace = True)

#fillna on the various missing values with reasonable substitutes
weather["avg_wind"].fillna(weather["avg_wind"].mean(), inplace = True)
weather['water_equiv_on_grd'].fillna(0, inplace = True)
weather['snowfall'].fillna(0, inplace = True)
weather["tavg"].fillna(((weather.tmax + weather.tmin) / 2), inplace = True)

# Merge Data

In [9]:
#merge arr_data_dist and precip_hour on 'arr_DateHour' and 'DateHour'
arr_precip_merge = arr_data_dist.merge(precip_hour, how = 'left', left_on = 'arr_DateHour', right_on = 'DateHour')

In [10]:
#merge arr_precip_merge and weather on 'date'
SLC_arrival_merge = arr_precip_merge.merge(weather, how = 'left', on = 'date')

In [12]:
SLC_arrival_merge.columns

Index(['scheduled_arr', 'carrier', 'origin', 'scheduled_elapsed',
       'actual_elapsed', 'arr_delay', 'faa_code', 'distance', 'arr_DateHour',
       'date', 'day_name', 'day_of_year', 'scheduled_hour', 'year', 'DateHour',
       'HourPrecip', 'avg_wind', 'precip', 'snowfall', 'tavg', 'tmax', 'tmin',
       'water_equiv_on_grd'],
      dtype='object')

In [13]:
#make a new df with only the info that will be used in the ML model, but keep categoricals for now
SLC_arr_ml_cat = SLC_arrival_merge[['carrier', 
                                    'scheduled_elapsed', 
                                    'distance', 
                                    'year',
                                    'day_name', 
                                    'day_of_year', 
                                    'scheduled_hour', 
                                    'HourPrecip', 
                                    'avg_wind',
                                    'precip', 
                                    'snowfall',
                                    'tavg',
                                    'tmax',
                                    'tmin', 
                                    'water_equiv_on_grd',
                                    'arr_delay']]

In [ ]:
#fillna the HourPrecip col with the mean for the column and create a column for "ontime"
SLC_arr_ml_cat['HourPrecip'].fillna(SLC_arr_ml_cat['HourPrecip'].mean(), inplace = True)


In [15]:
SLC_arr_ml_cat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2043281 entries, 0 to 2043280
Data columns (total 16 columns):
 #   Column              Dtype   
---  ------              -----   
 0   carrier             category
 1   scheduled_elapsed   int64   
 2   distance            int64   
 3   year                int64   
 4   day_name            object  
 5   day_of_year         int64   
 6   scheduled_hour      int64   
 7   HourPrecip          float64 
 8   avg_wind            float64 
 9   precip              float64 
 10  snowfall            float64 
 11  tavg                float64 
 12  tmax                int64   
 13  tmin                int64   
 14  water_equiv_on_grd  float64 
 15  arr_delay           int64   
dtypes: category(1), float64(6), int64(8), object(1)
memory usage: 251.4+ MB


In [ ]:
SLC_arr_ml_cat['ontime'] = (SLC_arr_ml_cat['arr_delay'] <= 0)

In [18]:
SLC_arr_ml_cat.head()

,carrier,scheduled_elapsed,distance,year,day_name,day_of_year,scheduled_hour,HourPrecip,avg_wind,precip,snowfall,tavg,tmax,tmin,water_equiv_on_grd,arr_delay,ontime
0,AA,197,1245,1988,Friday,1,11,0.0,9.17,0.0,0.0,17.0,27,7,0.5,22,False
1,AA,160,987,1988,Friday,1,11,0.0,9.17,0.0,0.0,17.0,27,7,0.5,9,False
2,AA,154,987,1988,Friday,1,12,0.0,9.17,0.0,0.0,17.0,27,7,0.5,21,False
3,AA,52,204,1988,Friday,1,15,0.0,9.17,0.0,0.0,17.0,27,7,0.5,-3,True
4,AA,48,188,1988,Friday,1,7,0.0,9.17,0.0,0.0,17.0,27,7,0.5,0,True


In [19]:
SLC_arr_ml_cat.to_csv("KSLC_arrivals_tidy.csv")